In [2]:
%pip install xgboost

  Using cached xgboost-2.1.1-py3-none-win_amd64.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.3/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.8/124.9 MB 1.5 MB/s eta 0:01:22
   ---------------------------------------- 1.0/124.9 MB 1.6 MB/s eta 0:01:19
    --------------------------------------- 1.6/124.9 MB 1.6 MB/s eta 0:01:15
    --------------------------------------- 1.8/124.9 MB 1.7 MB/s eta 0:01:15
    --------------------------------------- 2.1/124.9 MB 1.7 MB/s eta 0:01:15
    --------------------------------------- 2.6/124.9 MB 1.7 MB/s eta 0:01:13
    --------------------------------------- 2.9/124.9 MB 1.7 MB/s eta 0:01:13
   - -------------------------------------- 3.4/124.9 MB 1.7 MB/s eta 0:01:11
   - -------------------------------------- 3.7/124.9 MB 1.7 MB/s eta 0:01:11
   - ----------

In [3]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.metrics import r2_score, accuracy_score
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [5]:
#Load and explore new csv data after preprocessing
final_df_train = pd.read_csv("E:/Mine/Skin_cancer_detection/isic-2024-challenge/train_test_model.csv",  low_memory=False)
columns_to_drop = ['sex_nan', "tbp_lv_location_simple_Unknown", "anatom_site_general_nan"] 
final_df_train.drop(columns=columns_to_drop, inplace=True)
final_df_train.head()

,isic_id,target,patient_id,age_approx,clin_size_long_diam_mm,tbp_lv_A,tbp_lv_Aext,tbp_lv_B,tbp_lv_Bext,tbp_lv_C,...,tbp_lv_location_simple_Left Leg,tbp_lv_location_simple_Right Arm,tbp_lv_location_simple_Right Leg,tbp_lv_location_simple_Torso Back,tbp_lv_location_simple_Torso Front,anatom_site_general_anterior torso,anatom_site_general_head/neck,anatom_site_general_lower extremity,anatom_site_general_posterior torso,anatom_site_general_upper extremity
0,ISIC_0015670,0,IP_1235828,60,3.04,20.244422,16.261975,26.922447,23.954773,33.684638,...,0,0,1,0,0,0,0,1,0,0
1,ISIC_0015845,0,IP_8170065,60,1.10,31.712570,25.364740,26.331000,24.549290,41.219030,...,0,0,0,0,0,0,1,0,0,0
2,ISIC_0015864,0,IP_6724798,60,3.40,22.575830,17.128170,37.970460,33.485410,44.174920,...,0,0,0,1,0,0,0,0,1,0
3,ISIC_0015902,0,IP_4111386,65,3.22,14.242329,12.164757,21.448144,21.121356,25.746200,...,0,0,0,0,1,1,0,0,0,0
4,ISIC_0024200,0,IP_8313778,55,2.73,24.725520,20.057470,26.464900,25.710460,36.217980,...,0,0,0,0,1,1,0,0,0,0


In [6]:
import h5py
from sklearn.model_selection import train_test_split
# Load the metadata
# metadata_df = pd.read_csv('train-metadata.csv')
train_hd5 = "E:/Mine/Skin_cancer_detection/isic-2024-challenge/train-image.hdf5"

# Load the HDF5 file containing the images
hdf5_file = h5py.File(train_hd5, 'r')

# Get the image IDs (isic_id)
image_ids = list(hdf5_file.keys())

# Ensure the metadata is consistent with the image IDs
final_df_train = final_df_train[final_df_train['isic_id'].isin(image_ids)]

# Split the dataset into 80% for training and 20% for validation
train_ids, val_ids = train_test_split(final_df_train, test_size=0.2, random_state=42, stratify=final_df_train['target'])  # Stratified split to maintain class balance


In [7]:
print(f'train size: {train_ids.shape}, val size: {val_ids.shape}')

train size: (79, 98), val size: (20, 98)


In [14]:
# Convert all object columns to categorical type
train_ids[train_ids.select_dtypes(include=['object']).columns] = train_ids.select_dtypes(include=['object']).astype('category')
val_ids[val_ids.select_dtypes(include=['object']).columns] = val_ids.select_dtypes(include=['object']).astype('category')


In [ ]:
# Load your train/test dataframe into DMatrix
dtrain = xgb.DMatrix(
    train_ids.loc[:, train_ids.columns != "target"], label=train_ids["target"],
    enable_categorical=True
)
dtest = xgb.DMatrix(
    val_ids.loc[:, val_ids.columns != "target"], label=val_ids["target"],
    enable_categorical=True
)

# Catboost

In [24]:
%pip install catboost

   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 4.2 MB/s eta 0:00:24
    --------------------------------------- 1.3/101.7 MB 3.7 MB/s eta 0:00:27
    --------------------------------------- 2.1/101.7 MB 3.9 MB/s eta 0:00:26
   - -------------------------------------- 2.9/101.7 MB 3.8 MB/s eta 0:00:26
   - -------------------------------------- 3.7/101.7 MB 3.9 MB/s eta 0:00:26
   - -------------------------------------- 4.5/101.7 MB 3.9 MB/s eta 0:00:25
   -- ------------------------------------- 5.2/101.7 MB 3.9 MB/s eta 0:00:25
   -- ------------------------------------- 6.0/101.7 MB 3.9 MB/s eta 0:00:25
   -- ------------------------------------- 6.8/101.7 MB 3.9 MB/s eta 0:00:25
   -- ------------------------------------- 7.6/101.7 MB 3.9 MB/s eta 0:00:25
   --- ------------------------------------ 8.4/101.7 MB 3.9 MB/s eta 0:00:25
   --- ------------------------------------ 9.2/101.7 MB 3.8 MB/s eta 0